In [4]:
# Pasos:
#pip install mysql-connector-python
#pip install mysql

import pandas as pd
from datetime import timedelta
import mysql.connector
#import MySQLdb.connector
#from MySQLdb import connector


#### FUNCIONES
def fechas_consecutivas(lista_fechas, lista_categorias):
    categorias_unicas = set(lista_categorias) # set saca valores unicos de una lista
    resultado_por_categoria = {}

    for categoria in categorias_unicas:
        # Esta linea trae la lista de fechas que correspondan con el actual valor de categoria mediante una comprension de lista
        fechas_categoria = [fecha for fecha, cat in zip(lista_fechas, lista_categorias) if cat == categoria] # zip une dos listas en una tupla
        contador_actual = 0
        contador_maximo = 0

        for i in range(len(fechas_categoria) - 1): # -1 porque se omite la ultima fecha del array
            #fecha_actual = datetime.strptime(fechas_categoria[i], '%Y-%m-%d')
            #fecha_siguiente = datetime.strptime(fechas_categoria[i + 1], '%Y-%m-%d')
            fecha_actual = fechas_categoria[i]
            fecha_siguiente = fechas_categoria[i+1]
            
            suma_siete_dias = fecha_actual + timedelta(days=7)
            
            if suma_siete_dias == fecha_siguiente:
                contador_actual += 1
                contador_maximo = max(contador_maximo, contador_actual)
            else:
                contador_actual = 0
        
        resultado_por_categoria[categoria] = contador_maximo
    
    return pd.DataFrame(resultado_por_categoria.items(), columns=['Categoria', 'ContadorMaximo'])

In [12]:
# crawl the keys from a txt file
keys = pd.read_csv("C:/Users/Diego Torres/OneDrive - Manar Technologies SAS/Documentos compartidos/01_Proyectos/02_Externos/AlmacenesSI/02_ProyectoBI/576_ALMACENES_SI_PREDICCION_DEMANDA/2. Exploración/InsumosCliente/credencialesBD.txt")

# Creo variables para cada fila del DataFrame que contiene las credenciales de la bd
for index, row in keys.iterrows():
    variable_name = row['key']
    variable_value = row['value']
    globals()[variable_name] = variable_value


connection = mysql.connector.connect(host=host,
        port=port,
        user=user,
        password=password,
        database=database)

print(connection)

In [13]:
# Crear un cursor, el cursor sera el objeto que contiene las funciones para extraer las tablas de mysql a python
cursor = connection.cursor()

In [14]:
""" # Ejecutar consultas
## Exploracion tabla sales
cursor.execute("SELECT DISTINCT StoreId FROM analyticalways.sales")
results = cursor.fetchall()
print(sorted(results)) # a list
print(len(results)) """

' # Ejecutar consultas\n## Exploracion tabla sales\ncursor.execute("SELECT DISTINCT StoreId FROM analyticalways.sales")\nresults = cursor.fetchall()\nprint(sorted(results)) # a list\nprint(len(results)) '

In [28]:
## Numero de SKUs distintos con historia en cada uno de los 27 almacenes
cursor.execute("WITH temp AS (\
        SELECT DISTINCT\
         a.StoreId,\
         ProductId\
        FROM analyticalways.sales a\
        INNER JOIN analyticalways.store_information b\
          ON a.StoreId = b.StoreId\
        )\
        \
        SELECT\
         StoreId,\
         count(ProductId) counts\
        FROM temp\
        GROUP BY StoreId")
results = cursor.fetchall()

# Obtener los nombres de las columnas desde el cursor
column_names = [desc[0] for desc in cursor.description]
skus_per_store = pd.DataFrame(results, columns=column_names)
#df_ordenado_multi = df.sort_values(by=['Edad', 'Puntaje'], ascending=[True, False])
print(skus_per_store.sort_values(by='counts', ascending=False))
print(skus_per_store['counts'].sum())

    StoreId  counts
0   1010100  229398
2   1010800  136079
4   1011000  132290
1   1010400   68277
10  1014200   60654
8   1010900   43302
12  1012300   40583
9   1013700   35210
3   1011400   28168
5   1010200   27980
6   1010300   27510
11  1014500   10837
7   1010500    6740
13  1013100    5025
14  1014100    4474
15  1014300    3515
16  1014600     842
860884


In [25]:
## Base para usar en el modelamiento
# nota: comillas dobles en CONCAT arroja un error, usar sencillas
cursor.execute("WITH temp AS (\
        SELECT\
         Date,\
         CONCAT(a.StoreId,'_',ProductId) Store_SKU,\
         DATE_ADD(Date, INTERVAL (6 - WEEKDAY(Date)) DAY) AS UltimaFechaSemana,\
         AmountTaxesExcluded\
        FROM analyticalways.sales a\
        INNER JOIN analyticalways.store_information b\
          ON a.StoreId = b.StoreId\
        )\
        \
        SELECT\
         Store_SKU,\
         UltimaFechaSemana,\
         sum(AmountTaxesExcluded) Sales\
        FROM temp\
        WHERE AmountTaxesExcluded > 0\
        GROUP BY Store_SKU,UltimaFechaSemana\
        ORDER BY Store_SKU,UltimaFechaSemana")
results = cursor.fetchall()

# Obtener los nombres de las columnas desde el cursor
column_names = [desc[0] for desc in cursor.description]
sales_forecast = pd.DataFrame(results, columns=column_names)
print(' ')
print('Dimensiones:')
print(sales_forecast.shape)

 
Dimensiones:
(6585884, 3)


In [17]:
#codigo que genera el numero de registros consecutivos por cada serie
#test_2 = fechas_consecutivas(sales_forecast['UltimaFechaSemana'],sales_forecast['Store_SKU'])

In [30]:
# Calcular la tabla de frecuencia
test = sales_forecast['Store_SKU'].value_counts().reset_index() #equivalente a .table() de R
#test = test.assign(Continued_count = fechas_consecutivas('UltimaFechaSemana'))

# flag para mas de 20 puntos
test['apto'] = test['count'].apply(lambda x: 'Si' if x > 20 else 'No')
test['apto'].value_counts().reset_index()




,apto,count
0,No,794574
1,Si,65102


In [19]:
print(' ')
print('Dimensiones:')
print(test.shape)

 
Dimensiones:
(859873, 3)


In [20]:
print(test.head(20))

print(test.tail(20))

print(test[test['count']== 10])

                    Store_SKU  count apto
0   1010100_228456.0005.280CM    286   Si
1   1010100_234651.0005.148CM    282   Si
2      1011000_87614.0999.EE9    280   Si
3   1011000_228456.0005.280CM    280   Si
4   1011400_228456.0005.280CM    278   Si
5     1010200_226100.0005.EST    278   Si
6      1010100_87614.0999.EE9    278   Si
7   1010200_228456.0005.280CM    278   Si
8     1010100_100608.0005.EO2    278   Si
9   1010100_220760.1090.280CM    274   Si
10  1010100_220760.9010.280CM    274   Si
11  1011400_234651.0005.148CM    274   Si
12  1010100_234651.7930.148CM    272   Si
13  1010100_220760.0005.280CM    271   Si
14  1010200_220760.9010.280CM    271   Si
15  1010200_220760.1090.280CM    270   Si
16  1011000_234651.0005.148CM    269   Si
17  1012300_234651.0005.148CM    266   Si
18    1010100_209361.1000.UNI    265   Si
19    1010100_226100.0005.EST    264   Si
                       Store_SKU  count apto
859853   1010100_231978.9996.TXL      1   No
859854  1010800_251776.7150.

In [21]:
#ej serie con el maximo de datos: >280
sales_forecast[sales_forecast['Store_SKU'] == '1010100_228456.0005.280CM'].sort_values(by='UltimaFechaSemana', ascending=True).head(40)

,Store_SKU,UltimaFechaSemana,Sales
787585,1010100_228456.0005.280CM,2018-01-07,513475.75
787586,1010100_228456.0005.280CM,2018-01-14,420314.28
787587,1010100_228456.0005.280CM,2018-01-21,369285.72
787588,1010100_228456.0005.280CM,2018-01-28,40285.72
787589,1010100_228456.0005.280CM,2018-02-04,327432.15
787590,1010100_228456.0005.280CM,2018-02-11,308382.36
787591,1010100_228456.0005.280CM,2018-02-18,469622.77
787592,1010100_228456.0005.280CM,2018-02-25,559787.40
787593,1010100_228456.0005.280CM,2018-03-04,223210.10
787594,1010100_228456.0005.280CM,2018-03-11,903739.06


In [22]:
#ej serie con 1 dato
sales_forecast[sales_forecast['Store_SKU'] == '1014100_245196.3745.TXS'].sort_values(by='UltimaFechaSemana', ascending=True).head(40)

,Store_SKU,UltimaFechaSemana,Sales
6149149,1014100_245196.3745.TXS,2020-02-02,100831.93
6149150,1014100_245196.3745.TXS,2020-08-09,100831.93
6149151,1014100_245196.3745.TXS,2020-12-13,117638.66
6149152,1014100_245196.3745.TXS,2021-02-21,100833.61
6149153,1014100_245196.3745.TXS,2021-07-11,63021.01
6149154,1014100_245196.3745.TXS,2021-09-19,85709.24
6149155,1014100_245196.3745.TXS,2021-10-03,85709.24
6149156,1014100_245196.3745.TXS,2021-10-17,85709.24
6149157,1014100_245196.3745.TXS,2021-11-14,85709.24
6149158,1014100_245196.3745.TXS,2021-11-28,155453.78


In [23]:
#ej serie con 1 dato
sales_forecast[sales_forecast['Store_SKU'] == '1014600_271974.7686.TXL'].sort_values(by='UltimaFechaSemana', ascending=True).head(40)

,Store_SKU,UltimaFechaSemana,Sales
6586105,1014600_271974.7686.TXL,2023-06-18,87388.24


In [10]:
# Ejecutar consultas
""" cursor.execute("SELECT StoreId, count(*) rowss FROM analyticalways.actual_stock GROUP BY StoreId;")
results = cursor.fetchall()
print(pd.DataFrame(results)) """

In [20]:
# Ejecutar consultas
""" cursor.execute("SELECT\
                  YEAR(Date) anio,\
                  count(1) rowss\
                FROM analyticalways.sales\
                GROUP BY anio")
results = cursor.fetchall()
print(pd.DataFrame(results)) """

      0        1
0  2018  2794481
1  2019  2671189
2  2021  1937376
3  2022  1848100
4  2023  1031461
5  2020  1908559


In [26]:
# Ejecutar consultas
""" cursor.execute("SELECT * FROM analyticalways.sales")
results = cursor.fetchall() """

In [30]:
# Ejecutar consultas
""" cursor.execute("SELECT count(1) FROM analyticalways.sales")
results = cursor.fetchall()
print(results) """

[(12195488,)]


In [125]:

#### MAESTRA PRODUCTOS
cursor.execute("SELECT *\
                FROM analyticalways.product_information ;")
results = cursor.fetchall()
# Obtener los nombres de las columnas desde el cursor
column_names = [desc[0] for desc in cursor.description]
products = pd.DataFrame(results, columns=column_names)

#################
# Descriptivos
print(products.sample(10))
print(' ')
print('Summary:')
sales.info()
print(' ')
print('Descriptivos:')
print(products.describe().round(2).transpose())
print(' ')
print('Nulos por campo:')
print(products.isnull().sum())  # total de nulls por variable
print(' ')
print('Dimensiones:')
print(products.shape)

                        ProductId   
239336  183890.0860.TL             \
31487   137085.0005.T04             
447501  216630.1100.TM              
115512  162183.2054.147CM           
223832  181796.9996.TM              

                                            RootProductId      Cost     Price   
239336  183890.0860                                   ...  11469.00   5462.18  \
31487   137085.0005                                   ...   6678.31  11887.93   
447501  216630.1100                                   ...  10010.00  27310.92   
115512  162183.2054                                   ...   1970.00   2577.59   
223832  181796.9996                                   ...  11000.00  20588.24   

                                Level1                          Level2   
239336  VESTUARIO EXTERIOR              Top                             \
31487   UNIFORMES ESCOLARES             Top                              
447501  VESTUARIO EXTERIOR              Top                         

In [146]:

#### MAESTRA TIENDAS
cursor.execute("SELECT *\
                FROM analyticalways.store_information ;")
results = cursor.fetchall()
# Obtener los nombres de las columnas desde el cursor
column_names = [desc[0] for desc in cursor.description]
stores = pd.DataFrame(results, columns=column_names)

#################
# Descriptivos
print(stores.head(10))
print(' ')
print('Summary:')
stores.info()
print(' ')
print('Descriptivos:')
print(stores.describe().round(2).transpose())
print(' ')
print('Nulos por campo:')
print(stores.isnull().sum())  # total de nulls por variable
print(' ')
print('Dimensiones:')
print(stores.shape)

   StoreId                    Name Channel Email Description StoreType   
0  1010100                SI No. 1                                      \
1  1010200                SI No. 2                                       
2  1010300                 HOGARYS                                       
3  1010400                SI NORTE                                       
4  1010500          TIENDA VIRTUAL                                       
5  1010800  SI PALMIRA LLANOGRANDE                                       
6  1010900          SI UNICO STAFF                                       
7  1011000              SI CALIMA                                        
8  1011400        SI OUTLET CENTRO                                       
9  1012300                 ATELIER                                       

  ClassifficationLevels  
0                        
1                        
2                        
3                        
4                        
5                        
6  